# Associations

<img src="https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true">

La découverte d'associations est la recherche de liens entre plusieurs événements. Lorsque on étudie les événements qui ont lieu consécutivement à un ou plusieurs autres événements (analyse dans le temps) on parle de séquence.

Objectifs :<br>
•	Mettre en relation les différents produits et mieux comprendre les comportements de cross-selling des clients<br>
•	Quantifier l’existence des liaisons entre plusieurs produits<br>
•	Analyser le parcours des clients dans un magasin, sur un site Internet, …<br>
•	Mise en avant ou retrait d’un produit<br>


•	L’indice de **support** : mesure la fréquence d’apparition de A et B sur un même ticket (nombre de tickets avec A et B/nombre total de tickets)<br>
•	L’indice de **confiance** : probabilité d’apparition de B sur les tickets comprenant A (nombre de tickets avec A et B/nombre de tickets avec A)<br>
•	Le levier ou le **lift** : le poids relatif de cette association compte tenu de la fréquence d’apparition naturelle de B<br>


<img src="https://s3.ap-south-1.amazonaws.com/techleer/243.jpg">

## 0. Paramétrage

In [1]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [2]:
import datetime
now = datetime.datetime.now()
print(now)

2020-08-25 13:04:55.384126


In [3]:
# Installation librairie MLXTEND pour réaliser les associations
# Documentation : http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/
#!pip install mlxtend

In [4]:
import pandas as pd

In [5]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [6]:
import azureml.core
print("Version Azure ML :", azureml.core.VERSION)

Version Azure ML : 1.10.0


In [7]:
import pandas as pd
import logging
import os
import random

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

In [8]:
import os
subscription_id = os.environ.get("SUBSCRIPTION_ID", "ARENSEIGNER")
resource_group = os.environ.get("RESOURCE_GROUP", "ARENSEIGNER")
workspace_name = os.environ.get("WORKSPACE_NAME", "ARENSEIGNER")

from azureml.core import Workspace
try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print("OK")
except:
   print("Error: Workspace not found")

OK


## Création expérimentation

In [10]:
ws = Workspace.from_config()

experiment = Experiment(workspace=ws, name='Associations')

output = {}
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Workspace,workshopAML2020
Resource Group,workshopAML2020-rg
Location,westeurope
Experiment Name,Associations


In [11]:
run = experiment.start_logging(snapshot_directory=None)

## 1. Chargement des données


In [12]:
df=pd.read_csv('VentesFR.csv')

In [13]:
df.shape

(8555, 8)

In [14]:
df.head(20)

,CommandeID,Date,ClientID,ProduitID,Description,Quantite,Prix,Pays
0,536370,01/12/2010 08:45,12583.0,22728,ALARM CLOCK BAKELIKE PINK,24.0,3.75,France
1,536370,01/12/2010 08:45,12583.0,22727,ALARM CLOCK BAKELIKE RED,24.0,3.75,France
2,536370,01/12/2010 08:45,12583.0,22726,ALARM CLOCK BAKELIKE GREEN,12.0,3.75,France
3,536370,01/12/2010 08:45,12583.0,21724,PANDA AND BUNNIES STICKER SHEET,12.0,0.85,France
4,536370,01/12/2010 08:45,12583.0,21883,STARS GIFT TAPE,24.0,0.65,France
5,536370,01/12/2010 08:45,12583.0,10002,INFLATABLE POLITICAL GLOBE,48.0,0.85,France
6,536370,01/12/2010 08:45,12583.0,21791,VINTAGE HEADS AND TAILS CARD GAME,24.0,1.25,France
7,536370,01/12/2010 08:45,12583.0,21035,SET/2 RED RETROSPOT TEA TOWELS,18.0,2.95,France
8,536370,01/12/2010 08:45,12583.0,22326,ROUND SNACK BOXES SET OF4 WOODLAND,24.0,2.95,France
9,536370,01/12/2010 08:45,12583.0,22629,SPACEBOY LUNCH BOX,24.0,1.95,France


## 2. Préparation et transformation des données

In [15]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['CommandeID'], inplace=True)
df['CommandeID'] = df['CommandeID'].astype('str')
df = df[~df['CommandeID'].str.contains('C')]

In [16]:
basket = (df[df['Pays'] =="France"]
          .groupby(['CommandeID', 'Description'])['Quantite']
          .sum().unstack().reset_index().fillna(0)
          .set_index('CommandeID'))

In [17]:
basket

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
CommandeID,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537468,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [19]:
%%time
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

CPU times: user 16.3 ms, sys: 0 ns, total: 16.3 ms
Wall time: 23.3 ms


In [20]:
frequent_itemsets.head(20)

,support,itemsets
0,0.071429,(4 TRADITIONAL SPINNING TOPS)
1,0.096939,(ALARM CLOCK BAKELIKE GREEN)
2,0.102041,(ALARM CLOCK BAKELIKE PINK)
3,0.094388,(ALARM CLOCK BAKELIKE RED)
4,0.081633,(BAKING SET 9 PIECE RETROSPOT)
5,0.071429,(CHILDRENS CUTLERY DOLLY GIRL)
6,0.099490,(DOLLY GIRL LUNCH BOX)
7,0.096939,(JUMBO BAG RED RETROSPOT)
8,0.076531,(JUMBO BAG WOODLAND ANIMALS)
9,0.125000,(LUNCH BAG APPLE DESIGN)


In [21]:
frequent_itemsets.sort_values(by = 'support',  ascending = False)

,support,itemsets
22,0.188776,(RABBIT NIGHT LIGHT)
26,0.181122,(RED TOADSTOOL LED NIGHT LIGHT)
21,0.170918,(PLASTERS IN TIN WOODLAND ANIMALS)
18,0.168367,(PLASTERS IN TIN CIRCUS PARADE)
30,0.158163,(ROUND SNACK BOXES SET OF4 WOODLAND)
11,0.153061,(LUNCH BAG RED RETROSPOT)
14,0.142857,(LUNCH BOX WITH CUTLERY RETROSPOT)
33,0.137755,(SET/6 RED SPOTTY PAPER CUPS)
24,0.137755,(RED RETROSPOT MINI CASES)
19,0.137755,(PLASTERS IN TIN SPACEBOY)


## 3. Régles d'associations

In [22]:
%%time
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

CPU times: user 8.43 ms, sys: 36 µs, total: 8.47 ms
Wall time: 6.73 ms


### Liste des régles d'associations obtenues

In [23]:
rules.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061
5,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135
6,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.099490,0.125000,0.071429,0.717949,5.743590,0.058992,3.102273
7,(SPACEBOY LUNCH BOX),(DOLLY GIRL LUNCH BOX),0.125000,0.099490,0.071429,0.571429,5.743590,0.058992,2.101190
8,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN CIRCUS PARADE),0.137755,0.168367,0.089286,0.648148,3.849607,0.066092,2.363588
9,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN SPACEBOY),0.168367,0.137755,0.089286,0.530303,3.849607,0.066092,1.835747


### Tri par Lift

In [28]:
rules.sort_values(by = 'lift',  ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878,1
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181,1
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061,1
5,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135,1
25,(SET/6 RED SPOTTY PAPER PLATES),"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETROSPOT PAPER NAPKINS)",0.127551,0.102041,0.099490,0.780000,7.644000,0.086474,4.081633,1
20,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETROSPOT PAPER NAPKINS)",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959,2
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859,1
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383,1
23,(SET/6 RED SPOTTY PAPER CUPS),"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED SPOTTY PAPER PLATES)",0.137755,0.102041,0.099490,0.722222,7.077778,0.085433,3.232653,1
22,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED SPOTTY PAPER PLATES)",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796,2


### Liste des régles sur un antécédent particulier

In [29]:
rules[rules['antecedents'] == {'PLASTERS IN TIN CIRCUS PARADE'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
9,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN SPACEBOY),0.168367,0.137755,0.089286,0.530303,3.849607,0.066092,1.835747,1
11,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.168367,0.170918,0.102041,0.606061,3.545907,0.073264,2.104592,1


In [30]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383,1
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859,1
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878,1
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181,1
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061,1
5,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135,1
6,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.099490,0.125000,0.071429,0.717949,5.743590,0.058992,3.102273,1
7,(SPACEBOY LUNCH BOX),(DOLLY GIRL LUNCH BOX),0.125000,0.099490,0.071429,0.571429,5.743590,0.058992,2.101190,1
8,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN CIRCUS PARADE),0.137755,0.168367,0.089286,0.648148,3.849607,0.066092,2.363588,1
9,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN SPACEBOY),0.168367,0.137755,0.089286,0.530303,3.849607,0.066092,1.835747,1


### Sélection de régles

In [35]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.7) &
       (rules['lift'] > 2) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
20,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETROSPOT PAPER NAPKINS)",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.09949,0.9750,7.644000,0.086474,34.897959,2
21,"(SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY PAPER PLATES)",(SET/20 RED RETROSPOT PAPER NAPKINS),0.122449,0.132653,0.09949,0.8125,6.125000,0.083247,4.625850,2
22,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED SPOTTY PAPER PLATES)",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.09949,0.9750,7.077778,0.085433,34.489796,2


## 4. Régles pertinentes

In [36]:
%%time
rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.6) ]

CPU times: user 1.55 ms, sys: 303 µs, total: 1.85 ms
Wall time: 1.53 ms


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383,1
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859,1
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878,1
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181,1
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061,1
5,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135,1
6,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.099490,0.125000,0.071429,0.717949,5.743590,0.058992,3.102273,1
8,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN CIRCUS PARADE),0.137755,0.168367,0.089286,0.648148,3.849607,0.066092,2.363588,1
11,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.168367,0.170918,0.102041,0.606061,3.545907,0.073264,2.104592,1
12,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN SPACEBOY),0.170918,0.137755,0.104592,0.611940,4.442233,0.081047,2.221939,1


### Autres choix

In [37]:
mylift = 2
myconfidence = 0.7

In [38]:
rules[ (rules['lift'] >= mylift) &
       (rules['confidence'] >= myconfidence) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383,1
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859,1
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878,1
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181,1
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.102041,0.073980,0.783784,7.681081,0.064348,4.153061,1
5,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135,1
6,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.099490,0.125000,0.071429,0.717949,5.743590,0.058992,3.102273,1
13,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.137755,0.170918,0.104592,0.759259,4.442233,0.081047,3.443878,1
14,(SET/6 RED SPOTTY PAPER CUPS),(SET/20 RED RETROSPOT PAPER NAPKINS),0.137755,0.132653,0.102041,0.740741,5.584046,0.083767,3.345481,1
15,(SET/20 RED RETROSPOT PAPER NAPKINS),(SET/6 RED SPOTTY PAPER CUPS),0.132653,0.137755,0.102041,0.769231,5.584046,0.083767,3.736395,1


## 5. Export des régles pertinentes

### Export des régles dans un fichier CSV

In [43]:
dfrules=rules[ (rules['lift'] >= mylift) &
       (rules['confidence'] >= myconfidence) ]

dfrules.to_csv(r'myassociationrules.csv')

In [44]:
# Visualisation du fichier CSV créé
%ls myassociationrules.csv -l

-rwxrwxrwx 1 root root 4700 Aug 25 13:11 myassociationrules.csv*


### On loggue les valeurs et le fichier CSV des régles dans l'expérimentation Azure ML

In [45]:
# Logs des métriques l'expérimentation Azure ML
run.log('Lift Min', mylift)
run.log('Confidence Min', myconfidence)

In [46]:
# Sauvegarde du fichier CSV des régles pertinentes dans l'expérimentation
file_namecsv = 'myassociationrules.csv'
run.upload_file(name = file_namecsv, path_or_stream = file_namecsv)

### Accès à l'expérimentation Azure ML

In [47]:
experiment

Name,Workspace,Report Page,Docs Page
Associations,workshopAML2020,Link to Azure Machine Learning studio,Link to Documentation


In [48]:
 run.complete()

> Fin